In [1]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

In [22]:
def on_change(change):#si attiva quando cambio scelta dell'elemento da selezionare
    global allegato
    
    if change['type'] == 'change' and change['name'] == 'value':
        print ("Stai scegliendo %s" % change['new'])
        allegato=change['new']

def on_button_aggiorna(_):#definizione del bottone di riavvio
        #riavvia
    clear_output()
    print('Hai riavviato')
    display(Javascript('''var c = Jupyter.notebook.get_selected_index();

        IPython.notebook.execute_cells([c])'''))

def on_button_conferma(_):
        #crea nuova cella con immagine+altra ugualea questa
    global allegato
    clear_output()
    print('Hai scelto di allegare: '+allegato)
    open_attachment(allegato)
    #reload_cell_below()
    save_file_directory() 
    
def open_attachment(nome_allegato):#routine di scelta confermata
    print(nome_allegato)
    path_file=str(path_cartella_allegati_utente+ "/" + nome_allegato)
    ext = os.path.splitext(str(path_cartella_allegati_utente+ "/" + nome_allegato))[-1].lower()
    
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        file = open(path_file, "rb")
        image = file.read()
        w=widgets.Image(
            value=image,
            format=ext.replace('.', ''),
            width=300,
            height=400,
        )

        display(w)

def reload_cell_below():
    print("ricarica celle sotto")
    display(Javascript('''var c = Jupyter.notebook.get_selected_index();

            IPython.notebook.execute_cells([c+1])
            IPython.notebook.execute_cells([c+2])'''))
    #create_Button_main()
    
def risposta_libera_strutturata_main():
    #creare markdown
    aggiorna = widgets.Button(description='Aggiorna')
    conferma = widgets.Button(description='Conferma')

    numero_esercizio="X"#yaml
    #serve per aprire la cartella degli allegati dove l'utente li inserisce
    path_cartella_allegati_utente="allegati_"+numero_esercizio#yaml

    lista_allegati = os.listdir(path_cartella_allegati_utente)
    #serve per aprire il file dov'e l'elenco dei file allegati
    path_elencoallegati = 'elencoallegati.txt'
    elencoallegati=open(path_elencoallegati,'r')

    if(lista_allegati):#si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        global allegato
        allegato=lista_allegati[0]
        allegato_scelto_wid = widgets.Dropdown(
            options=lista_allegati,
            value=lista_allegati[0],
            description='Allegati:',
            disabled=False,
        )
        #allegato_scelto_wid=lista_allegati[0] da sistemare
        allegato_scelto_wid.observe(on_change)
        aggiorna.on_click(on_button_aggiorna)
        conferma.on_click(on_button_conferma)
        all_buttons = [aggiorna,allegato_scelto_wid,conferma]
        display(widgets.HBox(all_buttons))
    else:
        print("non ci sono allegati")
        aggiorna.on_click(on_button_aggiorna)
        display(aggiorna)

def save_file_directory():#non si avvia se non si seleziona un elemento
   # print("salvo il file nella cartella giusta, se non è gia stato inserito")
    #aggiungi codice
    #print("aggiungo nome file allegato ad un .txt contenente in ordine di inserimento i file")
    #apro il file con l'elenco degli allegati e mi creo una lista
    with open(path_elencoallegati, 'r') as f:
        attach_list = [line.strip() for line in f]
    attach_list.append(allegato)#aggiungo nuovo allegato
    #salvo l'allegato aggiunto
    f = open(path_elencoallegati,'w')
    for ele in attach_list:
        f.write(ele+'\n')

def on_bottone_elimina(_):
    print("on_bottone_elimina")
    elimina_cella_superiore()
    elimina_questa_cella()
    
def elimina_cella_superiore():
    display(Javascript('''var c = Jupyter.notebook.get_selected_index();
                                IPython.notebook.delete_cell([c-1])'''))
def elimina_questa_cella():
    display(Javascript('''var c = Jupyter.notebook.delete_cell();'''))
    
def on_bottone_libera_strutturata(_):
    print("on_bottone_libera_strutturata")
    aggiungi_cella_code_din('codice_bottone_elimina()')
    aggiungi_cella_code_din('risposta_libera_strutturata_main()')   
def on_bottone_testo(_):
    print("on_bottone_testo")
    aggiungi_cella_code_din('codice_bottone_elimina()')
    aggiungi_cella_raw_din()
    
def on_bottone_libera(_):
    print("on_bottone_libera")
    aggiungi_cella_code_din('codice_bottone_elimina()')
    aggiungi_cella_raw_din()
    
def aggiungi_cella_raw_din():
    s_all = 'Hai creato cella raw alla richiesta'
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text(' ''' + s_all + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index])'''))
    return 

def aggiungi_cella_code_din(code):
    display(Javascript('''var c = Jupyter.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index])'''))
    return

def codice_bottone_elimina():
    bottone_elimina = widgets.Button(description="Elimina")
    bottone_elimina.on_click(on_bottone_elimina)
    print("vuoi eliminare cella sopra?")
    display(bottone_elimina)

def loader_main():
    bottone_libera_strutturata = widgets.Button(description="Libera Strutturata",tooltip="allega un file")
    bottone_testo = widgets.Button(description="Testo",tooltip="crea filemarkdown")
    bottone_libera= widgets.Button(description="Libera")
    bottone_libera_strutturata.on_click(on_bottone_libera_strutturata)
    bottone_testo.on_click(on_bottone_testo)
    bottone_libera.on_click(on_bottone_libera)

    all_buttons = [bottone_libera,bottone_testo,bottone_libera_strutturata]

    print("Con che tipo di modalità vuoi rispondere?")
    display(widgets.HBox(all_buttons))
numero_esercizio="X"#yaml
    #serve per aprire la cartella degli allegati dove l'utente li inserisce
path_cartella_allegati_utente="allegati_"+numero_esercizio#yaml
allegato=""

In [23]:
def is_subseq(s, subs):
    """
    bool: verifica se subs è sottosequenza di s.
    """
    found = 0
    pos_r = 0
    while pos_r < len(s):
        if s[pos_r] == subs[found]:
            found += 1
            if found >= len(subs):
                return True
        pos_r += 1
    return False

def evaluation_format(answ, pt_green,pt_red):
    pt_blue=pt_red-pt_green
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"


# Legend of the possible sequence types:
dictionary_of_types = {
      "SC": ("implemented", "<b>strettamente crescente</b>"),
      "ND": ("implemented", "<b>non-decrescente</b>"),
      "SD": ("implemented", "<b>strettamente decrescente</b>"),
      "NC": ("implemented", "<b>non-crescente</it>"),
       "V": ("implemented", "<b>a V</b> <it>(prima giù e poi sù)</it>"),
       "A": ("implemented", "<b>ad A</b> (prima sù e poi giù)</it>"),
      "SV": ("implemented", "<b>a V stretto</b> <it>(prima strettamente giù e poi strettamente sù)</it>"),
      "SA": ("implemented", "<b>ad A stetto</b> <it>(prima strettamente sù e poi strettamente giù)</it>"),
  "ZigZag": ("implemented", "<b>a Zig-Zag</b> <it>(primo passo a crescere e poi alterna ad ogni passo)</it>"),
  "ZagZig": ("implemented", "<b>a Zag-Zig</b> <it>(primo passo a calare e poi alterna ad ogni passo)</it>"),
"ZigZagEQ": ("implemented", "<b>a Zig-Zag debole</b> <it>(primo passo a crescere e poi alterna ad ogni passo, con valori consecutivi che possono essere uguali)</it>"),
"ZagZigEQ": ("implemented", "<b>a Zag-Zig debole</b> <it>(primo passo a calare e poi alterna ad ogni passo, con valori consecutivi che possono essere uguali)</it>"),
"132-free": ("not yet done", "<b>dal mondo delle permutazioni pattern free per un infinità di problemi in FPT</b>"),
     "...": ("not thought of yet", "<b>???</b>")
}

def Latex_type(seq_type):
    return dictionary_of_types[seq_type][1].replace("_", "\_")


def is_seq_of_type(s, name_s, seq_type):
    """
    valuta se la sequenza di interi s, di nome name_s, è di tipo seq_type (vedi tabella subito sopra).
    valore di ritorno (bool, NO_cert_string):
       In caso affermativo il bool ritornato come prima componente è True e la seconda componente è None.
       Altrimenti il bool è False e viene restituita una stringa che riporta una violazione puntuale alla proprietà richiesta.
    """
    if len(s) > 1:
        already_went_down = s[1] < s[0]
        already_went_up = s[1] > s[0]
    for i in range(1,len(s)):
        if s[i] < s[i-1]:
            already_went_down = True
            if seq_type=="V" and already_went_up:
                return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo ${Latex_type('V')}$ poichè ${LaTexVarName(name_s)}[${i-1}$] = {s[i-2]} $<$ {s[i-1]} $= {LaTexVarName(name_s)}[${i}$] > {LaTexVarName(name_s)}[${i+1}$] =$ {s[i]}.")
            if seq_type in {"SC","ND"} or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 1) or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 0):
                return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo ${Latex_type(seq_type)}$ poichè ${LaTexVarName(name_s)}[${i}$] = {s[i-1]} $>$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
        if s[i] > s[i-1]:
            already_went_up = True
            if seq_type=="A" and already_went_down:
                return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type('A')} poichè ${LaTexVarName(name_s)}[${i-1}$] =$ {s[i-2]} $>$ {s[i-1]} $= {LaTexVarName(name_s)}[${i}$] < {LaTexVarName(name_s)}[${i+1}$] =$ {s[i]}.")
            if seq_type in {"SD","NC"} or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 1) or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 0):
                return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $<$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
        if s[i] == s[i-1] and seq_type in {"SC","SD","SV","SA","ZigZag","ZagZig"}:
            return (0,f"La sequenza ${LaTexVarName(name_s)}$ non è di tipo {Latex_type(seq_type)} poichè ${LaTexVarName(name_s)}[${i}$] =$ {s[i-1]} $=$ {s[i]} $= {LaTexVarName(name_s)}[${i+1}$]$.")
    return (1,None)

def LaTexVarName(var_name):
    return var_name.replace("_", "\_")


def is_subseq_of_type(s, name_s, subs, name_subs, subs_type, pt_green, pt_red, forced_ele_pos = None, start_banned_interval = None, end_banned_interval = None):
    """
    Verifica se subs, una sequenza di interi di nome name_subs, è sequenza di tipo subs_type (vedi tabella) e sottosequenza della sequenza s.
    Se forced_ele_pos != None è richiesto che subs contenga l'elemento s[forced_ele_pos].
    Se start_banned_interval,end_banned_interval != None è richiesto che subs eviti il sottointervallo bandito di s.
    Restituisce una stringa contenete la valutazione del certificato subs immesso dallo studente, a tale scopo i parametri pt_green e pt_red mentre pt_blue=pt_red-pt_green è fatto implicito.
    """
    submission_string = f"Hai inserito il certificato ${LaTexVarName(name_subs)}={subs}$."
    submission_string += f"<br>L'istanza era data da ${LaTexVarName(name_s)}={s}$.<br>"

    if not is_seq_of_type(subs, "subs", subs_type)[0]:
        return submission_string + evaluation_format("No", pt_green,pt_red) + is_seq_of_type(subs, "subs", subs_type)[1]
    if start_banned_interval != None or end_banned_interval != None:
        assert start_banned_interval != None and end_banned_interval != None
        if forced_ele_pos != None:
            assert forced_ele_pos < start_banned_interval or forced_ele_pos > end_banned_interval
            if forced_ele_pos > end_banned_interval:
                forced_ele_pos -= end_banned_interval 
        aux = s[:start_banned_interval-1] +s[end_banned_interval:]
    if not is_subseq(s, subs):
        return submission_string + f"{evaluation_format('No', pt_green,pt_red)}" + f"La sequenza ${LaTexVarName(name_subs)}$ proposta non è sottosequenza di ${LaTexVarName(name_s)}$."
    if forced_ele_pos != None:
        forced_ele_0basedpos = forced_ele_pos
        found_magic_point = False
        for guess_0basedpos_in_subs in range(len(subs)):
            if subs[guess_0basedpos_in_subs] == s[forced_ele_0basedpos]:
                if is_subseq(s[:forced_ele_0basedpos], subs[:guess_0basedpos_in_subs]) and is_subseq(s[forced_ele_0basedpos:], subs[guess_0basedpos_in_subs:]):
                    found_magic_point = False
        if not found_magic_point:
            return submission_string + f"{evaluation_format('No', pt_green,pt_red)}" + f"La sequenza $LaTexVarName({name_subs})$ proposta non è sottosequenza di $LaTexVarName({name_s}) che ne includa l'elemento in posizione {forced_ele_pos}$."
        
    return submission_string + f"{evaluation_format('Si', pt_green,pt_red)}"

def eval_coloring(s, name_s, col, name_col, subs_type, pt_green=2, pt_red=15):
    """
    Verifica se col, una sequenza di interi positivi di nome name_col, offre una colorazione degli elementi nella sequenza s,
    di nome name_s, tale che restringendo l'attenzione ai soli elementi di qualsivoglia colore, la sottosequenza di essi
    sia sequenza di tipo subs_type (vedi tabella).
    Restituisce una stringa contenete la valutazione del certificato col immesso dallo studente, a tale scopo i parametri pt_green e pt_red mentre pt_blue=pt_red-pt_green è fatto implicito.
    """
    submission_string = f"Hai inserito il certificato ${LaTexVarName(name_col)}={col}$."
    submission_string += f"<br>L'istanza era data da ${LaTexVarName(name_s)}={s}$.<br>"

    for c in col:
        subs = [s[i] for i in range(len(s)) if col[i] == c]
        if not is_seq_of_type(subs, "subs", subs_type)[0]:
            return submission_string + f"{evaluation_format('No', pt_green,pt_red)}" + f"Checking the subsequence of the elements colored with {c} within ${LaTexVarName(name_s)}$, that is {subs} ... " + is_seq_of_type(subs, "subs", subs_type)[1]        
    return submission_string + f"{evaluation_format('Si', pt_green,pt_red)}"
    


## Esercizio \[60 pts\]
(poldo) Ricerca di sottosequenze strettamente crescenti di massima lughezza.

Si consideri la seguente sequenza di numeri naturali:

In [24]:
s = [11, 24, 18, 12, 14, 31, 38, 58, 15, 36, 59, 19, 42, 29, 22, 56, 20, 17, 41, 13, 33, 35, 21, 16, 39, 49, 60, 54, 23]

In [25]:
print(s)

[11, 24, 18, 12, 14, 31, 38, 58, 15, 36, 59, 19, 42, 29, 22, 56, 20, 17, 41, 13, 33, 35, 21, 16, 39, 49, 60, 54, 23]



__Richieste__:


1. __[10 pts]__ Trovare una sottosequenza $s1$ strettamente crescente di $s$ che sia la più lunga possibile.

In [32]:
 codice_bottone_elimina() 

vuoi eliminare cella sopra?


Button(description='Elimina', style=ButtonStyle())

In [34]:
 risposta_libera_strutturata_main() 

In [33]:
 codice_bottone_elimina() 

vuoi eliminare cella sopra?


Button(description='Elimina', style=ButtonStyle())

In [30]:
loader_main()
#libera=aggiungi cella testo raw
#Testo= aggiungi cella markdown
#Liberastr=allega attachment

Con che tipo di modalità vuoi rispondere?


on_bottone_libera


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

on_bottone_testo


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

on_bottone_libera_strutturata


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
opt = [11,12,14,15,19,20,33,35,39,49,60]
opt2 = [11,12,14,15,19,20,33,35,39,49,54]
feas = [24,36,56,60]
wrong1 = [-3]
wrong2 = [5,49]

In [ ]:
print(f"s={s}")
display(Markdown(is_subseq_of_type(s, "s", opt, "opt", "SC", pt_green=1, pt_red=10)))
print(f"Nota: Di fatto opt concretizzerà anche i 9 punti blu come potrai convincerti autonomamente grazie al certificato di NO eventualmente prodotto in ultima domanda in fondo all'esercizio.")
display(Markdown(is_subseq_of_type(s, "s", opt2, "opt2", "SC", pt_green=1, pt_red=10)))
print(f"Nota: Se opt ne merita 10 allora anche opt2 è opt e ne merita altrettanti. Le soluzioni ottime ad un problema possono essere più di una!")
display(Markdown(is_subseq_of_type(s, "s", feas, "feas", "SC", pt_green=1, pt_red=10)))
print(f"Nota: Ma questi 9 punti blu non è sempre detto ti saranno infine assegnati: durante l'esame avrai a disposizione solo dei validatori (Re Artù) di cui avvalerti, noi non forniamo i solutori per non spoilerare l'esercizio ed il tuo divertimento (nonchè il senso dell'esame).")
display(Markdown(is_subseq_of_type(s, "s", wrong1, "wrong1", "SC", pt_green=1, pt_red=10)))
print(f"Nota: Ma questi 9 punti blu non devi considerarli sempre in cassaforte (se non hai i certificati di NO la soluzione potrebbe non essere ottima, come ad esempio in questo caso di feas. Dopo la consegna otterresti solo il punto verde.")
display(Markdown(is_subseq_of_type(s, "s", wrong2, "wrong2", "SC", pt_green=1, pt_red=10)))
print(f"Nota: I validatori (Re Artù) ti assistono evitandoti errori stupidi o l'aver frainteso la consegna in vari modi possibili.")

2. __[10 pts]__ Trovare una sottosequenza $s2$ strettamente decrescente di $s$ che sia la più lunga possibile.


In [ ]:
#loader2

In [ ]:
subs2 = [5,49]

In [ ]:
display(Markdown(is_subseq_of_type(s, "s", subs2, "subs2", "SD", pt_green=1, pt_red=10, forced_ele_pos=None, start_banned_interval=None, end_banned_interval=None)))

3. __[10 pts]__ Trovare la più lunga sottosequenza crescente che includa l'elemento di valore 7

In [ ]:
print(s[18-1])

In [ ]:
subs3 = [5,49]

In [ ]:
display(Markdown(is_subseq_of_type(s, "s", subs3, "subs3", "SC", pt_green=1, pt_red=10, forced_ele_pos=18, start_banned_interval=None, end_banned_interval=None)))

4. __[10 pts]__ Una sequenza è detta una _V-sequenza_ se cala fino ad un certo punto, e da lì in poi cresce sempre. Trovare la più lunga V-sequenza che sia una sottosequenza della sequenza data

In [ ]:
subs4 = [5,49]

In [ ]:
display(Markdown(is_subseq_of_type(s, "s", subs4, "subs4", "V", pt_green=1, pt_red=10)))

5. __[20 pts]__ Qual è il minor numero possibile di colori _C_ per colorare gli elementi della sequenza in input in modo che, per ogni colore, la sottosequenza degli elementi di quel colore sia monotona non crescente? Specificare per ogni elemento il colore (come colori, usare i numeri da 1 a _C_)


In [ ]:
recall_s = [11, 24, 18, 12, 14, 31, 38, 58, 15, 36, 59, 19, 42, 29, 22, 56, 20, 17, 41, 13, 33, 35, 21, 16, 39, 49, 60, 54, 23]
col_opt  = [ 1,  2,  2,  2,  3,  4,  5,  6,  4,  5,  7,  5,  6,  6,  6,  7,  6,  5,  7,  3,  7,  8,  7,  5,  9, 10, 11, 11,  8]
col_feas = [ 1,  2,  2,  2,  3,  4,  5,  6,  4,  5,  7,  5,  6,  6,  6,  7,  6,  5,  7,  3,  7,  8,  8,  7, 10, 12, 14, 15,  8]
col_bad1 = [ 1,  2,  2,  2,  3,  4,  5,  4,  4,  5,  7,  5,  5,  6,  6,  7,  6,  5,  7,  3,  7,  8,  7,  5, 16, 20, 11, 11,  8]
col_bad2 = [-1,  2,  2,  2,  3,  4,  5,  6,  4,  5,  7,  5,  5,  6,  6,  7,  6,  5,  7,  3,  7,  8,  7,  5, 16, 20, 11, 11,  8]
col_bad3 = [ 0,  2,  2,  2,  3,  4,  5,  6,  4,  5,  7,  5,  5,  6,  6,  7,  6,  5,  7,  3,  7,  8,  7,  5, 16, 20, 11, 11,  8]


In [ ]:
display(Markdown(eval_coloring(s, "s", col_opt, "col_opt", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_feas, "col_feas", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_bad1, "col_bad1", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_bad2, "col_bad2", "SD", pt_green=2, pt_red=15)))


In [ ]:
display(Markdown(eval_coloring(s, "s", col_opt, "col_opt", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_feas, "col_feas", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_bad1, "col_bad1", "SD", pt_green=2, pt_red=15)))
display(Markdown(eval_coloring(s, "s", col_bad2, "col_bad2", "SD", pt_green=2, pt_red=15)))
